In [ ]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [ ]:
location = "hugo" # "urban" "amsterdam" "jungle" 
# if we make a new location / or project we need to do the following:
# create a csv labels file
# create a csv labels prop file
# create the new directories that hold the new train and eval samples
#
# for example the new project `hugo`
# in ..serval/dataset/classlabels/csv_files we need 
# class_labels_indices_hugo.csv
# class_labels_indices_hugo_prop.csv (copy of class_labels_indices_hugo.csv)
# in this csv we need to add a column `target_ind` and put all zeros, this is a placeholder of the propertions per class
# train_hugo_new and eval_hugo_new in serval/dataset


In [ ]:
# inside docker goto right place 
os.chdir("/tf/serval/serval")
# set relvant full paths
# /tf is the tensorflow jupyter container mapping
path_original_tf_records = "dataset/audioset_v1_embeddings/"
path_new_tf_records = "dataset/"
path_class_labels = "dataset/classlabels/csv_files/"
#os.getcwd()

In [ ]:
## Paden naar originele tf-records: bal_train, unbal_train en eval
bal_train_data_pattern = path_original_tf_records + "bal_train/*.tfrecord"
unbal_train_data_pattern = path_original_tf_records + "unbal_train/*.tfrecord"
eval_data_pattern = path_original_tf_records + "eval/*.tfrecord"

## Paden voor nieuwe tf-records (train en test)
train_pattern_new = path_new_tf_records + "train_" + location + "_new"
eval_pattern_new = path_new_tf_records + "eval_" + location + "_new"

In [ ]:
## Boolean die aangeeft of alle examples worden geschreven naar een tf-record
## Bij false: upsamplen/downsamplen adhv proporties
write_all_examples = True
## proportie train/test
pct_train = 0.6

In [ ]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []
label_combinations = []

In [ ]:
## Open csv-bestanden met class labels
## Oorspronkelijke csv-file google audioset class labels
mid_to_label_old = pd.read_csv(path_class_labels + "class_labels_indices_original_audioset.csv", sep=",")

## csv-file met labels (urban/jungle/amsterdam)
new_label_file = "class_labels_indices_" + location + ".csv"
mid_to_label_new = pd.read_csv(path_class_labels + new_label_file, sep=";")
## mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_urban.csv', sep=";")

## Naam bestand om proporties naartoe te schrijven (of om te gebruiken bij sampelen)
## Als deze nog niet bestaat: maak een kopietje van mid_to_labels_new met de naam die hieronder staat
name_label_prop_file = "class_labels_indices_" + location + "_prop.csv"
label_proportions = pd.read_csv(path_class_labels + name_label_prop_file, sep=";")

In [ ]:
mid_to_label_new

In [ ]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1 or write_all_examples):
        return True
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [ ]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1 or write_all_examples):
        return False
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:    
            return True
        else:
            return False
    return False

In [ ]:
#### Find new label for each of old label (527 -> x)

def getNewLabel(label_old):
    
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [ ]:
def createExample(example_old, new_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [ ]:
## Deze code haalt voor een label alle (grand)parents op
## Momenteel worden deze functies niet meer gebruikt.
## Maar schroom je niet om eens te kijken hoe deze prachtige recursieve functie werkt!

def getParents(label, parents):
    
    parent = int(mid_to_label_new.loc[mid_to_label_new['index']==label, 'parent'])
    
    if parent==999:
        return parents
    else:
        parents.append(parent)
        return getParents(parent, parents)
    return parent

def getLabelsWithParents(labels):
    total_labels = labels
    for label in labels:
        parents = getParents(label, [])
        total_labels.extend(parents)
    return(set(total_labels))

In [ ]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readWriteTfRecord(tfrecords_filename, prefix):
    
    ## Maak 2 writers (train en test) die nieuw tf-records schrijft
    
    # windows 
    #record_name = tfrecords_filename.split('\\')[-1]
    # linux
    record_name = tfrecords_filename.split('/')[-1]
    ## Let op: hij doet moeilijk over AUX.tfrecord. Dus prefix A bij UX mag niet
    #if(str(prefix+record_name_=="AUX.tfrecord"):
    #    record_name="UXx.tfrecord"
        
    ## Nieuwe filenames: 1 voor train en 1 voor eval
    train_file_new = str(train_pattern_new + '/' + prefix + record_name)
    eval_file_new = str(eval_pattern_new + '/' + prefix + record_name)

    train_writer = tf.python_io.TFRecordWriter(train_file_new)
    eval_writer = tf.python_io.TFRecordWriter(eval_file_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## Als enkele (grand)parents ontbreken, voeg ze toe
        #new_labels = getLabelsWithParents(new_labels)
        ## If any of the labels left (for our model), write new tf-record
        
        if (len(new_labels) > 0 and checkIfNewExample(new_labels)):
        #if(len(new_labels) > 0):
            ## Add to all labels
            all_labels.extend(new_labels)
            label_combinations.append(new_labels)
   
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            rand = random.random()
            if rand <= pct_train:
                train_writer.write(sequence_example.SerializeToString())
            else:
                eval_writer.write(sequence_example.SerializeToString())
        
            ## Check of er nog een example moet worden gemaakt (upsamplen)
            if(checkIfExtraExample(new_labels)):
                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(sequence_example.SerializeToString())
                else:
                    eval_writer.write(sequence_example.SerializeToString())
        
                all_labels.extend(new_labels)
                label_combinations.append(new_labels)
            print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value)
                  + ' labels : ' + str(new_labels))

    #print("nr new examples: " + str(nr_new_examples))
    
    # Only write if any examples created

    if nr_new_examples > 0:
        train_writer.close()
        eval_writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(train_file_new)
        os.remove(eval_file_new)

In [ ]:
## Maak nieuwe tf-records van bal_train
# make sure the new directory location exists train_amsterdam_new
# ISSUE amsterdam has new labels? csv not ; separated

files = gfile.Glob(bal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'B')

In [ ]:
## Maak nieuwe tf-records van unbal_train
# LETOP deze is groot en duurt lang, voor tests niet nodig

files = gfile.Glob(unbal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'U')

In [ ]:
## Maak nieuwe tf-records van eval

files = gfile.Glob(eval_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'E')

In [ ]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(label_proportions.loc[label_proportions['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

In [ ]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(label_proportions.shape[0])] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]
    
    for i in label_list:
        label_names.append(str(label_proportions.loc[label_proportions['index']==i]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

In [ ]:
print("sum occurences: " + str(sum_occur))
print("ind occ: ")
print(ind_occur)

In [ ]:
## UP-/DOWN-SAMPLEN

## Alle code hierna is alleen om uit te rekenen wat de proporties zijn
## Het werkt als volgt:
## Van het totale aantal voorkomens van een label (bv 11.000), wordt het gewenste afgetrokken:
## bv 11.000-2.000 = 9.000
## Dit aantal examples moet geskipt worden, maar dan alleen als het label alleen voorkomt
## Daarom wordt de target proportie van het totale individuutjes (target_ind) als volgt berekend:
## Stel ind_occ (aantal keer alleen voorkomend) is 10.000, waarvan we er 9.000 niet willen hebben.
## (10.000-9.000)/10.000 = 0.1 is dan de kans dat een example wordt geschreven.

In [ ]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens en aantal voorkomens alleen
label_proportions["total"] = 0
label_proportions["ind_total"] = 0
label_proportions["proportion"] = 0
label_proportions["target_ind"] = 0

for i in range(mid_to_label_new.shape[0]):
    label_proportions.loc[i,"total"] = cnt_labels[i]
    label_proportions.loc[i,"ind_total"] = ind_occur[i]   
    label_proportions.loc[i,"proportion"] = cnt_labels[i]/sum_occur
    label_proportions.loc[i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]

In [ ]:
## Schrijf bestandje weg
label_proportions.to_csv(name_label_prop_file, sep = ";")

In [ ]:
label_proportions

In [ ]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readTfRecord(tfrecords_filename):
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        print(example)
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
       
        #if(labels[0]!=11):
        #    print(labels)
        break;

In [ ]:
file = 'tfrecords/train_jungle_new/gun_train.tfrecord'

In [ ]:
readTfRecord(file)